In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

#import dmitrys loading code
import sys; sys.path.append('../../libs/rna-seq-tsne/')
import rnaseqTools

import pickle
import os.path
import pandas as pd
import numpy as np
import warnings
import anndata
from scipy.io import mmread
from scipy import sparse

from tools import remove_rare_genes

/usr/local/lib/python3.6/dist-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


# PMBC data

In [2]:
def load_PMBC_dataset(folder,genename_file='genes.tsv'):
    counts = mmread(folder + 'matrix.mtx')
    counts = sparse.csr_matrix(counts).T
    genes = pd.read_csv(folder + genename_file, header=None, sep='\t')
    genes = genes.values[:,1].squeeze().astype(str)
    return counts,genes

In [3]:
%%time
#50sec
folder = 'datasets/33k_pbmc/'
label = '33k PBMC (v1)'
counts,genes=load_PMBC_dataset(folder)
counts,genes=remove_rare_genes(counts,genes,minimum_detected_cells_per_gene=5)

counts = sparse.csr_matrix(counts)

dataset = dict(counts=counts,genes=genes,label=label,folder=folder)
pickle.dump(dataset, open(folder+'preprocessed.pickle', "wb"), protocol=4)

Of 20678 total genes, returning 16809 genes that are detected in 5 or more cells.
Output shape: (33148, 16809)
CPU times: user 44.6 s, sys: 777 ms, total: 45.4 s
Wall time: 45.8 s


# Negative control data
### 10X v2 - Svensson 2017 (technical control)

In [4]:
%%time
folder = 'datasets/10x/'
file = folder + 'svensson_chromium_control.h5ad'

raw_input = anndata.read(file)

for i,sample_id in enumerate([20311, 20312]):
    sample_input = raw_input[raw_input.obs.query('sample == "%u"' % (sample_id)).index]
    label = '2k Chromium control Svensson 2017 (10x v2) - sample %u' % (i+1)
    
    counts_pd = sample_input.to_df()
    genes = np.array(counts_pd.columns)
    counts = counts_pd.to_numpy()
    counts,genes=remove_rare_genes(counts,genes,minimum_detected_cells_per_gene=5)

    counts = sparse.csr_matrix(counts)

    dataset = dict(counts=counts,genes=genes,label=label,folder=folder,protocol='10X')
    pickle.dump(dataset, open(folder+'sample%u_preprocessed.pickle' % (i+1), "wb"), protocol=4)

Of 20647 total genes, returning 13025 genes that are detected in 5 or more cells.
Output shape: (2000, 13025)
Of 21411 total genes, returning 13239 genes that are detected in 5 or more cells.
Output shape: (2000, 13239)
CPU times: user 1.74 s, sys: 41.4 ms, total: 1.78 s
Wall time: 1.78 s


# inDrop

### Klein 2015 (one technial and various biological controls)

In [5]:
paths =['datasets/indrop/GSM1599501_K562_pure_RNA.csv']
ns = ['1k']
labelsshort = ['K562_pure_RNA']

In [6]:
def pickle_klein_data(path,labelshort,n):
    label = '%s %s Klein 2015 (inDrop)' % (n,labelshort)
    df = pd.read_csv(path,index_col=0)
    genes = df.index
    counts = df.values.T
    counts,genes = remove_rare_genes(counts,genes,minimum_detected_cells_per_gene=5)
    counts = sparse.csr_matrix(counts)
    dataset = dict(counts=counts,genes=genes,folder=path,label=label,protocol='inDrop')
    picklefile = '%s_preprocessed.pickle' % (path)
    pickle.dump(dataset, open(picklefile, "wb"), protocol=4)

In [7]:
%%time
## pickle-prepares all klein datasets
for p,n,l in zip(paths,ns,labelsshort):
    pickle_klein_data(p,l,n)

Of 25266 total genes, returning 25025 genes that are detected in 5 or more cells.
Output shape: (953, 25025)
CPU times: user 2.52 s, sys: 121 ms, total: 2.64 s
Wall time: 2.82 s


# MicrowellSeq

### Han 2018 (biological controls)

In [8]:
%%time
## Preproc files for ES cells (works for CJ7 deep, CJ7 normal and E14)
files_raw = ['datasets/microwellseq/GSM2906413_EmbryonicStemCell_dge.txt.gz']
separators = [' ']

### iterate over files and make pickles for each tissue type

for i,(file_raw,sep) in enumerate(zip(files_raw,separators)):

    file = file_raw.split('/')[-1]
    cellline = file.split('_')[1]
    
    counts, genes, cells = rnaseqTools.sparseload(file_raw, sep=sep)

    counts = counts.toarray()
    print(file,counts.shape)
    
    #remove low depth cells
    depths = np.array(np.sum(counts,axis=1)).flatten()
    minimum_depth = 500
    cells = cells[depths>minimum_depth]
    counts = counts[depths>minimum_depth,:]
    print('Of',len(depths),'cells, returning',sum(depths>minimum_depth),'cells that have a depth larger than', minimum_depth)
    print('New shape:', counts.shape)    
    
    counts,genes = remove_rare_genes(counts,genes,minimum_detected_cells_per_gene=5)
    counts = sparse.csr_matrix(counts)

    dataset = dict(counts=counts,genes=genes,cells=cells,folder=file_raw,
                   label='Han 2018 %s control (MicrowellSeq)' % (cellline),
                   protocol='microwellSeq')
    
    print(dataset)
    picklefile = '%s_preprocessed.pickle' % (file_raw)
    pickle.dump(dataset, open(picklefile, "wb"), protocol=4)

..................... done
GSM2906413_EmbryonicStemCell_dge.txt.gz (9994, 20682)
Of 9994 cells, returning 9994 cells that have a depth larger than 500
New shape: (9994, 20682)
Of 20677 total genes, returning 15069 genes that are detected in 5 or more cells.
Output shape: (9994, 15069)
{'counts': <9994x15069 sparse matrix of type '<class 'numpy.float64'>'
	with 6762689 stored elements in Compressed Sparse Row format>, 'genes': array(['0610007P14Rik', '0610009B22Rik', '0610009L18Rik', ..., 'n-R5s151',
       'n-R5s2', 'n-R5s88'], dtype='<U14'), 'cells': array(['EmbryonicStemCells_1.CTCGCAATCAACAACCTA',
       'EmbryonicStemCells_1.CCGCTAATTCCAGTTGCC',
       'EmbryonicStemCells_1.CTCGCACACAAGACACCC', ...,
       'EmbryonicStemCells_1.CGAGTACCAGACAGATGG',
       'EmbryonicStemCells_1.CGGCAGCTGAAAGGCTGC',
       'EmbryonicStemCells_1.CTGTGTGAGATCGTCCCG'], dtype=object), 'folder': 'datasets/microwellseq/GSM2906413_EmbryonicStemCell_dge.txt.gz', 'label': 'Han 2018 EmbryonicStemCell control (